# Polars

- [Website](https://pola.rs/)
- [Github repo](https://github.com/pola-rs/polars)


[Pandas](https://pandas.pydata.org/) (the Python package) is a great easy-to-use library for exploring and apply transformations to dataframes. However, one of its limitations is that it doesn't perform well with larger datasets. Pandas stores data in-memory and has a memory limit of 100GB. 

For use cases where you're dealing with data beyond the 100GB memory threshold, this is where Polars shines. Polars, which was built with Rust and uses Apache Arrow's columnar format, is said to execute common operations [10-100 times faster](https://blog.jetbrains.com/dataspell/2023/08/polars-vs-pandas-what-s-the-difference/#:~:text=As%20you%20can%20see%2C%20Polars,out%2Dof%2Dmemory%20errors.) than Pandas . With Polars' Lazy DataFrame, we can also handle larger-than-memory data beyond the 100GB threshold. 

```
pip install polars
```

In [1]:
import polars as pl

df = pl.read_csv("data/airbnb_nyc_2019.csv")

Polars is able to read several file formats including:
- csv 
- json 
- parquet 

You can check out the full list of Polars functions [here](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/functions.html). Some of these functions are identical to what Pandas offers (e.g., head, max, mean, n_unique).

In [2]:
df.head(2)

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
i64,str,i64,str,str,str,f64,f64,str,i64,i64,i64,str,f64,i64,i64
2539,"""Clean & quiet …",2787,"""John""","""Brooklyn""","""Kensington""",40.64749,-73.97237,"""Private room""",149,1,9,"""2018-10-19""",0.21,6,365
2595,"""Skylit Midtown…",2845,"""Jennifer""","""Manhattan""","""Midtown""",40.75362,-73.98377,"""Entire home/ap…",225,1,45,"""2019-05-21""",0.38,2,355


### Select

The `select` method allows you to select the columns that you care about. You can also chain methods together as shown below. 

In [3]:
df.select(pl.col("id", "name")).tail()

id,name
i64,str
36484665,"""Charming one b…"
36485057,"""Affordable roo…"
36485431,"""Sunny Studio a…"
36485609,"""43rd St. Time …"
36487245,"""Trendy duplex …"


### Filter

The `filter` method really reminds me of [PySpark's filter()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html). 

In [4]:
import datetime as datetime 

df.filter(
    (pl.col("number_of_reviews") > 100) & (pl.col("price") < 300)
).shape

(2890, 16)

### Casting to a different datatype

Casting a series to another datatype took a while to figure out but it's pretty straight-forward. Also very similar to PySpark's syntax.

In [5]:
df.drop_nulls(
    subset="last_review"
).with_columns(
    pl.col("last_review").str.to_datetime("%Y-%m-%d")
).select(
    pl.col("last_review")
).head(5)

last_review
datetime[μs]
2018-10-19 00:00:00
2019-05-21 00:00:00
2019-07-05 00:00:00
2018-11-19 00:00:00
2019-06-22 00:00:00


### Grouping By

In [6]:
df.group_by(
    "neighbourhood_group"
).agg(
    pl.col("id").count().alias("number_of_listings")
)

neighbourhood_group,number_of_listings
str,u32
"""Brooklyn""",20104
"""Bronx""",1091
"""Queens""",5666
"""Manhattan""",21661
"""Staten Island""",373


In [7]:
df.null_count()

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,21,0,0,0,0,0,0,0,0,10052,10052,0,0


### Exploring the Lazy vs Eager API

The Lazy API evaluates a query on an as-needed basis. It reduces its load on memory by using optimizations like:

- Predicate pushdown: applies filters as early as possible, so that you only read in the rows that are required for the query 
- Projection pushdown: only loads the columns that are needed

In almost all cases, the Lazy API is favoured over the Eager API, with the exception of scenarios where you're not familiar with the data and want to do exploratory data analysis.

To call the Lazy API, you can use `scan_csv` (instead of `read_csv`). 

In [8]:
df_lazy = (
    pl.scan_csv("data/airbnb_nyc_2019.csv")
    .with_columns(pl.col("neighbourhood_group").str.to_uppercase())
    .filter(pl.col("price") > 500)
    .collect()
)
df_lazy.shape

(1044, 16)

💡 The `collect()` method needs to be appended at the end. This is what executes the query. 

❗ The output of the query above is not cached. So if we want to re-use this same operation downstream, it'll be re-computed from scratch. 

If you're working with a massive dataset, you can execute your queries on a subset of the data using the `fetch()` method. The `n_rows` parameter indicates how many rows from the dataset you'd like to use. 

In [9]:
df_lazy = (
    pl.scan_csv("data/airbnb_nyc_2019.csv")
    .with_columns(pl.col("neighbourhood_group").str.to_uppercase())
    .filter(pl.col("price") > 500)
    .fetch(n_rows=300)
)
df_lazy.shape

(2, 16)

### Streaming API

The Lazy API also offers a streaming option. This is super useful when you're dealing with a larger-than-memory dataset and don't want to execute your query in one big batch. With streaming, you can execute the query in smaller batches which makes it easier to process extremely large datasets. 

You can configure the size of each batch using `set_streaming_chunk_size` as shown below:

In [10]:
pl.Config.set_streaming_chunk_size(100)
df_lazy = (
    pl.scan_csv("data/airbnb_nyc_2019.csv")
    .select(pl.col("neighbourhood_group", "price"))
    .with_columns(pl.col("neighbourhood_group").str.to_uppercase())
    .filter(pl.col("price") > 500)
    .collect(streaming=True)
)
df_lazy.head()

neighbourhood_group,price
str,i64
"""BROOKLYN""",800
"""MANHATTAN""",575
"""BROOKLYN""",650
"""BROOKLYN""",599
"""MANHATTAN""",2000


❗ Streaming API is a new Polars feature and is still under construction. Streaming is only offered for a handle of operators including:
- `filter`
- `slice`
- `with_columns`
- `select`
- `group_by`
- `join`
- `sort`
- `scan_csv` 
- `explode` and `melt` 